### Demand Data Processing

In [163]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import sparse
import datetime
from datetime import datetime

In [164]:
data = pd.read_csv('Data/demand_1dayahead.csv')
data.head()

,DAYSAHEAD,TARGETDATE,FORECASTDEMAND,CARDINALPOINT,CP_TYPE,CP_ST_TIME,CP_END_TIME,F_Point,FORECAST_TIMESTAMP
0,1,2018-01-02T00:00:00Z,24920,1F,F,30,30,NaN,2018-01-01T08:46:43Z
1,1,2018-01-02T00:00:00Z,25250,1A,P,100,230,NaN,2018-01-01T08:46:43Z
2,1,2018-01-02T00:00:00Z,23390,03,F,300,300,NaN,2018-01-01T08:46:43Z
3,1,2018-01-02T00:00:00Z,22480,1B,T,430,730,Om,2018-01-01T08:46:43Z
4,1,2018-01-02T00:00:00Z,37190,2F,P,800,900,NaN,2018-01-01T08:46:43Z


In [165]:
data.columns = ['days_ahead', 'date', 'demand', 'cardinal', 'cp_type', 'cp_st_time', 'cp_en_time', 'f_point', 'forecast_timestamp']
data.head(5)

,days_ahead,date,demand,cardinal,cp_type,cp_st_time,cp_en_time,f_point,forecast_timestamp
0,1,2018-01-02T00:00:00Z,24920,1F,F,30,30,NaN,2018-01-01T08:46:43Z
1,1,2018-01-02T00:00:00Z,25250,1A,P,100,230,NaN,2018-01-01T08:46:43Z
2,1,2018-01-02T00:00:00Z,23390,03,F,300,300,NaN,2018-01-01T08:46:43Z
3,1,2018-01-02T00:00:00Z,22480,1B,T,430,730,Om,2018-01-01T08:46:43Z
4,1,2018-01-02T00:00:00Z,37190,2F,P,800,900,NaN,2018-01-01T08:46:43Z


In [166]:
data['date'] = pd.to_datetime(data.date, format= "%Y-%m-%dT%H:%M:%SZ")
data.head()

,days_ahead,date,demand,cardinal,cp_type,cp_st_time,cp_en_time,f_point,forecast_timestamp
0,1,2018-01-02,24920,1F,F,30,30,NaN,2018-01-01T08:46:43Z
1,1,2018-01-02,25250,1A,P,100,230,NaN,2018-01-01T08:46:43Z
2,1,2018-01-02,23390,03,F,300,300,NaN,2018-01-01T08:46:43Z
3,1,2018-01-02,22480,1B,T,430,730,Om,2018-01-01T08:46:43Z
4,1,2018-01-02,37190,2F,P,800,900,NaN,2018-01-01T08:46:43Z


In [167]:
data = data.drop(['days_ahead', 'cp_type', 'cp_st_time', 'cp_en_time', 'f_point', 'forecast_timestamp'], axis=1)
data.head()

,date,demand,cardinal
0,2018-01-02,24920,1F
1,2018-01-02,25250,1A
2,2018-01-02,23390,03
3,2018-01-02,22480,1B
4,2018-01-02,37190,2F


In [168]:
for i in data.cardinal.unique():
    data['demand_' + i] = data.loc[data.cardinal == i]['demand']

data.head()

,date,demand,cardinal,demand_1F,demand_1A,demand_03,demand_1B,demand_2F,demand_2A,demand_2B,...,demand_DP,demand_4B,demand_4C,demand_L04,demand_3C,demand_4A,demand_1S,demand_04,demand_4,demand_3
0,2018-01-02,24920,1F,24920.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-02,25250,1A,NaN,25250.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-02,23390,03,NaN,NaN,23390.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-02,22480,1B,NaN,NaN,NaN,22480.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-02,37190,2F,NaN,NaN,NaN,NaN,37190.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
data_new = data.drop(['demand', 'cardinal'], axis=1)
data_new.head()

,date,demand_1F,demand_1A,demand_03,demand_1B,demand_2F,demand_2A,demand_2B,demand_3B,demand_DP,demand_4B,demand_4C,demand_L04,demand_3C,demand_4A,demand_1S,demand_04,demand_4,demand_3
0,2018-01-02,24920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-02,NaN,25250.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-02,NaN,NaN,23390.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-02,NaN,NaN,NaN,22480.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-02,NaN,NaN,NaN,NaN,37190.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
data_new =data_new.fillna(0)
data_new.head()

,date,demand_1F,demand_1A,demand_03,demand_1B,demand_2F,demand_2A,demand_2B,demand_3B,demand_DP,demand_4B,demand_4C,demand_L04,demand_3C,demand_4A,demand_1S,demand_04,demand_4,demand_3
0,2018-01-02,24920.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2018-01-02,0.0,25250.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2018-01-02,0.0,0.0,23390.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2018-01-02,0.0,0.0,0.0,22480.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2018-01-02,0.0,0.0,0.0,0.0,37190.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [171]:
data_new = data_new.groupby('date').sum()

In [172]:
data_new.head(2)

,demand_1F,demand_1A,demand_03,demand_1B,demand_2F,demand_2A,demand_2B,demand_3B,demand_DP,demand_4B,demand_4C,demand_L04,demand_3C,demand_4A,demand_1S,demand_04,demand_4,demand_3
date,,,,,,,,,,,,,,,,,,
2018-01-02,24920.0,25250.0,23390.0,22480.0,37190.0,38930.0,40050.0,39630.0,44500.0,37530.0,26690.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-01-03,25520.0,25570.0,23890.0,23200.0,39730.0,40380.0,39870.0,39610.0,46730.0,39850.0,27830.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [173]:
data_new = data_new[data_new.columns[:11]]

In [174]:
data_new.head()

,demand_1F,demand_1A,demand_03,demand_1B,demand_2F,demand_2A,demand_2B,demand_3B,demand_DP,demand_4B,demand_4C
date,,,,,,,,,,,
2018-01-02,24920.0,25250.0,23390.0,22480.0,37190.0,38930.0,40050.0,39630.0,44500.0,37530.0,26690.0
2018-01-03,25520.0,25570.0,23890.0,23200.0,39730.0,40380.0,39870.0,39610.0,46730.0,39850.0,27830.0
2018-01-04,27230.0,27610.0,26270.0,25650.0,41210.0,41930.0,41470.0,40820.0,46740.0,40150.0,28210.0
2018-01-05,28980.0,29260.0,27280.0,26040.0,41500.0,42020.0,41460.0,40590.0,46520.0,39550.0,28960.0
2018-01-06,29150.0,29400.0,26560.0,24620.0,34690.0,37150.0,37060.0,36150.0,43920.0,37100.0,28710.0


In [175]:
data_new.index.rename('index', inplace=True)

In [176]:
data_new.head()

,demand_1F,demand_1A,demand_03,demand_1B,demand_2F,demand_2A,demand_2B,demand_3B,demand_DP,demand_4B,demand_4C
index,,,,,,,,,,,
2018-01-02,24920.0,25250.0,23390.0,22480.0,37190.0,38930.0,40050.0,39630.0,44500.0,37530.0,26690.0
2018-01-03,25520.0,25570.0,23890.0,23200.0,39730.0,40380.0,39870.0,39610.0,46730.0,39850.0,27830.0
2018-01-04,27230.0,27610.0,26270.0,25650.0,41210.0,41930.0,41470.0,40820.0,46740.0,40150.0,28210.0
2018-01-05,28980.0,29260.0,27280.0,26040.0,41500.0,42020.0,41460.0,40590.0,46520.0,39550.0,28960.0
2018-01-06,29150.0,29400.0,26560.0,24620.0,34690.0,37150.0,37060.0,36150.0,43920.0,37100.0,28710.0


In [177]:
data_new['date'] = data_new.index
data_new['date'] = pd.to_datetime(data_new.date, format="%Y-%m-%d")
data_new.head()

,demand_1F,demand_1A,demand_03,demand_1B,demand_2F,demand_2A,demand_2B,demand_3B,demand_DP,demand_4B,demand_4C,date
index,,,,,,,,,,,,
2018-01-02,24920.0,25250.0,23390.0,22480.0,37190.0,38930.0,40050.0,39630.0,44500.0,37530.0,26690.0,2018-01-02
2018-01-03,25520.0,25570.0,23890.0,23200.0,39730.0,40380.0,39870.0,39610.0,46730.0,39850.0,27830.0,2018-01-03
2018-01-04,27230.0,27610.0,26270.0,25650.0,41210.0,41930.0,41470.0,40820.0,46740.0,40150.0,28210.0,2018-01-04
2018-01-05,28980.0,29260.0,27280.0,26040.0,41500.0,42020.0,41460.0,40590.0,46520.0,39550.0,28960.0,2018-01-05
2018-01-06,29150.0,29400.0,26560.0,24620.0,34690.0,37150.0,37060.0,36150.0,43920.0,37100.0,28710.0,2018-01-06


In [178]:
start_date = '2020-06-30'
end_date = '2021-06-24'

In [179]:
mask = (data_new['date'] >= start_date) & (data_new['date'] <= end_date)
data_new= data_new.loc[mask]
display(data_new)

,demand_1F,demand_1A,demand_03,demand_1B,demand_2F,demand_2A,demand_2B,demand_3B,demand_DP,demand_4B,demand_4C,date
index,,,,,,,,,,,,
2020-06-30,37195.0,35142.0,0.0,33435.0,49230.0,50342.0,52028.0,51105.0,0.0,53472.0,43582.0,2020-06-30
2020-07-01,41447.0,38303.0,0.0,35350.0,51443.0,53277.0,53047.0,51060.0,0.0,53183.0,42478.0,2020-07-01
2020-07-02,41469.0,38869.0,0.0,36059.0,52404.0,54407.0,55445.0,51625.0,0.0,54020.0,43294.0,2020-07-02
2020-07-03,41873.0,38467.0,0.0,35405.0,49783.0,51465.0,52426.0,49147.0,0.0,48797.0,40515.0,2020-07-03
2020-07-04,38153.0,35590.0,0.0,32283.0,44119.0,47829.0,48995.0,42886.0,0.0,44888.0,38668.0,2020-07-04
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-20,40168.0,37710.0,0.0,34440.0,45782.0,49549.0,52080.0,48306.0,0.0,53292.0,43121.0,2021-06-20
2021-06-21,41467.0,39028.0,0.0,36313.0,58829.0,60576.0,61823.0,59634.0,0.0,59022.0,45649.0,2021-06-21
2021-06-22,43968.0,41895.0,0.0,39294.0,56942.0,56159.0,53552.0,49916.0,0.0,58694.0,47602.0,2021-06-22


In [180]:
data_new = data_new.drop(['date','demand_DP', 'demand_03'], axis=1)
data_new = data_new.div(1000)
data_new.head()

,demand_1F,demand_1A,demand_1B,demand_2F,demand_2A,demand_2B,demand_3B,demand_4B,demand_4C
index,,,,,,,,,
2020-06-30,37.195,35.142,33.435,49.230,50.342,52.028,51.105,53.472,43.582
2020-07-01,41.447,38.303,35.350,51.443,53.277,53.047,51.060,53.183,42.478
2020-07-02,41.469,38.869,36.059,52.404,54.407,55.445,51.625,54.020,43.294
2020-07-03,41.873,38.467,35.405,49.783,51.465,52.426,49.147,48.797,40.515
2020-07-04,38.153,35.590,32.283,44.119,47.829,48.995,42.886,44.888,38.668


In [181]:
data_new.to_csv('Data/demand_forecast.csv')